In [1]:
import sys
sys.path.append("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/ASAN_01_Repeatition_Counter")

In [4]:
# ===============================================================
# check_dataset_yaml.py
# YOLO 학습 전에 dataset.yml 설정 검증 스크립트
# ===============================================================

import yaml  # YAML 파일 읽기용
from pathlib import Path  # 경로 처리용
import os  # 파일 존재 여부 확인용

# ---------------------------------------------------------------
# 1️⃣ 주요 필드 검사 함수
# ---------------------------------------------------------------
def check_dataset_yaml(yaml_path: str):
    """
    dataset.yml 구조를 검증하고 각 경로의 존재 여부를 확인합니다.
    Args:
        yaml_path (str): YOLO 학습용 dataset.yml 파일 경로
    """
    yaml_path = Path(yaml_path)  # Path 객체로 변환

    # YAML 파일 존재 여부 확인
    if not yaml_path.exists():
        print(f"❌ dataset.yml 파일이 존재하지 않습니다: {yaml_path}")
        return

    # YAML 로드
    with open(yaml_path, "r", encoding="utf-8") as f:
        try:
            data = yaml.safe_load(f)  # YAML 파싱
        except yaml.YAMLError as e:
            print(f"❌ YAML 파싱 오류: {e}")
            return

    # 필수 키 존재 여부 확인
    required_keys = ["train", "val", "keypoints", "skeleton"]
    for k in required_keys:
        if k not in data:
            print(f"❌ 누락된 필드: {k}")
            return

    # -----------------------------------------------------------
    # 2️⃣ 경로 유효성 검사
    # -----------------------------------------------------------
    for split in ["train", "val"]:
        p = Path(data[split])
        if not p.exists():
            print(f"⚠️ {split} 경로가 존재하지 않습니다: {p}")
        else:
            num_txt = len(list(p.rglob("*.txt")))
            print(f"✅ {split}: {num_txt}개의 라벨 파일 존재")

    # -----------------------------------------------------------
    # 3️⃣ keypoints, skeleton 필드 검사
    # -----------------------------------------------------------
    keypoints = data["keypoints"]
    skeleton = data["skeleton"]

    if not isinstance(keypoints, list) or not all(isinstance(k, str) for k in keypoints):
        print("❌ keypoints 필드는 문자열 리스트 형식이어야 합니다.")
        return
    if not isinstance(skeleton, list) or not all(isinstance(s, list) for s in skeleton):
        print("❌ skeleton 필드는 리스트의 리스트 형식이어야 합니다.")
        return

    print(f"✅ keypoints 개수: {len(keypoints)}개")
    print(f"✅ skeleton 연결선 개수: {len(skeleton)}개")

    # -----------------------------------------------------------
    # 4️⃣ 추가 검사 (옵션)
    # -----------------------------------------------------------
    if "nc" in data:
        print(f"✅ 클래스 수 (nc): {data['nc']}")
    if "names" in data:
        print(f"✅ 클래스 이름: {data['names']}")
    if "pose" in data:
        print(f"✅ pose 모델 설정 존재: {data['pose']}")

    print("🎯 dataset.yml 검증 완료 ✅")


# ---------------------------------------------------------------
# 5️⃣ 실행 예시
# ---------------------------------------------------------------
if __name__ == "__main__":
    yml_path = "/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/ASAN_01_Repeatition_Counter/data/dataset_validated.yml"
    check_dataset_yaml(yml_path)


❌ 누락된 필드: keypoints


In [2]:
from functions.render_skeleton_video import render_skeleton_video
import pandas as pd
from pathlib import Path

# ------------------------------------------------------------
# 설정
# ------------------------------------------------------------
targets = ("AI_dataset/N02",)
csv_path = Path("/workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/ASAN_01_Repeatition_Counter/data/metadata_backup.csv")

# ------------------------------------------------------------
# metadata 로드 및 필터링
# ------------------------------------------------------------
df = pd.read_csv(csv_path)
mask = df["frame_path"].str.contains("|".join(targets), case=False, na=False)
df_target = df[mask]

print(f"[INFO] 대상 데이터 {len(df_target)}개 선택됨 (필터: {targets})")

# ------------------------------------------------------------
# Skeleton 렌더링 실행
# ------------------------------------------------------------
for i, row in df_target.iterrows():
    frame_dir = Path(row["frame_path"])
    json_dir = Path(row["keypoints_path"])

    # ✅ CSV에 이미 있는 mp4_path 사용
    out_mp4 = Path(row["mp4_path"])
    out_mp4.parent.mkdir(parents=True, exist_ok=True)

    if not frame_dir.exists() or not json_dir.exists():
        print(f"[WARN] 경로 없음 → frame: {frame_dir}, json: {json_dir}")
        continue

    print(f"\n[{i+1}/{len(df_target)}] 🎬 {frame_dir.name}")
    try:
        render_skeleton_video(
            frame_dir=str(frame_dir),
            json_dir=str(json_dir),
            out_mp4=str(out_mp4),
            fps=30,
            model_type="coco17",       # 12 keypoints skeleton
            flip_horizontal=False
        )
    except Exception as e:
        print(f"[ERROR] render 실패: {frame_dir.name} ({e})")

print("\n✅ 전체 Skeleton Overlay 렌더링 완료!")


[INFO] 대상 데이터 96개 선택됨 (필터: ('AI_dataset/N02',))

[97/96] 🎬 diagonal__biceps_curl


diagonal__biceps_curl: 100% 344/344 [00:05<00:00, 67.45frame/s]


✅ Skeleton overlay 완료 → /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/3_MP4/AI_dataset/N02/N02_Treatment/diagonal__biceps_curl.mp4

[98/96] 🎬 diagonal__hip_extension


diagonal__hip_extension: 100% 465/465 [00:09<00:00, 48.89frame/s]


✅ Skeleton overlay 완료 → /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/3_MP4/AI_dataset/N02/N02_Treatment/diagonal__hip_extension.mp4

[99/96] 🎬 diagonal__hip_flexion_extension


...

✅ Skeleton overlay 완료 → /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/3_MP4/AI_dataset/N02/N02_Ward/lateral__shoulder_abduction.mp4

[191/96] 🎬 lateral__shoulder_flexion


lateral__shoulder_flexion: 100% 360/360 [00:11<00:00, 32.29frame/s]


✅ Skeleton overlay 완료 → /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/3_MP4/AI_dataset/N02/N02_Ward/lateral__shoulder_flexion.mp4

[192/96] 🎬 lateral__slr


lateral__slr: 100% 336/336 [00:12<00:00, 27.82frame/s]


✅ Skeleton overlay 완료 → /workspace/nas203/ds_RehabilitationMedicineData/IDs/Kimjihoo/3_project_HCCmove/data/3_MP4/AI_dataset/N02/N02_Ward/lateral__slr.mp4

✅ 전체 Skeleton Overlay 렌더링 완료!
